In [16]:
import json

class InventoryEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Inventory) or isinstance(obj, Backpack):
            return obj.to_json()  # to_json() method for custom classes
        return super().default(obj)  # default method for other types

# Base class definition for RPG items
class Inventory:
    def __init__(self, name, description="", rarity="common"):
        self.name = name
        self.description = description
        self.rarity = rarity
        self._ownership = ""  # Initially, no one owns the item

    def pick_up(self, character_name):
        self._ownership = character_name
        return f"{character_name} picked up {self.name}."

    def throw_away(self):
        self._ownership = ""
        return f"{self.name} has been thrown away."

    def use(self):
        if self._ownership:
            return f"{self.name} is used."
        else:
            return f"{self.name} cannot be used as it has no owner."

    def __str__(self):
        ownership_status = self._ownership if self._ownership else "No ownership."
        return f"Item: {self.name}, Description: {self.description}, Rarity: {self.rarity}, Ownership: {ownership_status}"

    def __eq__(self, other):
        return isinstance(other, Inventory) and self.name == other.name and self.rarity == other.rarity

    def to_json(self):
        return {
            'name': self.name,
            'description': self.description,
            'rarity': self.rarity,
            'ownership': self._ownership,
        }

    @classmethod
    def from_json(cls, data):
        instance = cls(name=data['name'], description=data['description'], rarity=data['rarity'])
        instance._ownership = data.get('ownership', "")  # Set ownership if provided
        return instance

class Backpack:
    def __init__(self, owner=None):
        self.owner = owner 
        self.items = []     # List to store items

    def add_item(self, item):
        if item not in self.items:
            item.pick_up(self.owner)  # Update ownership when item is picked up
            self.items.append(item)
            return f"{item.name} has been added to the backpack."
        return f"{item.name} is already in the backpack."

    def remove_item(self, item):
        if item in self.items:
            self.items.remove(item)
            item.throw_away()  # Reset ownership when item is removed
            return f"{item.name} has been removed from the backpack."
        return f"{item.name} is not in the backpack."

    def view_item(self, item):
        if item in self.items:
            return str(item)  # Call the __str__ method of the Inventory class
        return f"{item.name} is not in the backpack."

    def view_all_items(self):
        if not self.items:
            return "The backpack is empty."
        return [str(item) for item in self.items]  # Return a list of item strings

    def view_items_by_type(self, item_type):
        filtered_items = [str(item) for item in self.items if isinstance(item, item_type)]
        if not filtered_items:
            return f"There are no items of type {item_type.__name__} in the backpack."
        return filtered_items
    
    def to_json(self):
        return {
            'owner': self.owner,
            'items': [item.to_json() for item in self.items]  # Convert each item to JSON format
        }

    @classmethod
    def from_json(cls, data):
        instance = cls(owner=data['owner'])
        for item_data in data['items']:
            item_type = item_data.get('type')
            if item_type == 'Weapon':
                item = Weapon.from_json(item_data)
            elif item_type == 'Shield':
                item = Shield.from_json(item_data)
            # Add the deserialized item to the backpack
            instance.add_item(item)
        return instance

    def __iter__(self):
        return iter(self.items)

    def __contains__(self, item):
        return item in self.items

# Subclass: Weapon
class Weapon(Inventory):
    def __init__(self, name, description, rarity, damage, weapon_type):
        super().__init__(name, description, rarity)
        self.damage = damage
        self.weapon_type = weapon_type
        self.is_equipped = False
        self.attack_mod = 1.0 if rarity in ["common", "uncommon", "epic"] else 1.15

    def __str__(self):
        base_info = super().__str__() + f", Weapon Type: {self.weapon_type}, Damage: {self.damage}, Attack Modifier: {self.attack_mod}"
        if self.rarity == "legendary":
            legendary_art = (
                "\n🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟\n"
                "        L E G E N D A R Y       \n"
                "🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟\n"
                f"{self.name}: A weapon of unimaginable power!\n"
            )
            return base_info + legendary_art
        return base_info

    def to_json(self):
        data = super().to_json()  # Get base class JSON
        data.update({
            'damage': self.damage,
            'weapon_type': self.weapon_type,
            'type': 'Weapon'  # Add item type for identification
        })
        return data

    @classmethod
    def from_json(cls, data):
        instance = cls(
            name=data['name'],
            description=data['description'],
            rarity=data['rarity'],
            damage=data['damage'],
            weapon_type=data['weapon_type']
        )
        instance._ownership = data.get('ownership', "")  # Set ownership if provided
        return instance


# Subclass for Master Sword
class MasterSword(Weapon):
    def attack_move(self):
        return f"{self.name} swings with legendary might!"

#Subclass Shield
class Shield(Inventory):
    def __init__(self, name, description, rarity, defense, is_broken=False):
        super().__init__(name, description, rarity)
        self.defense = defense
        self.is_broken = is_broken

    def to_json(self):
        data = super().to_json()  # Get base item details
        data.update({
            'type': 'Shield',  # Specify the type for JSON
            'defense': self.defense,
            'is_broken': self.is_broken,
        })
        return data

    @classmethod
    def from_json(cls, data):
        instance = cls(
            name=data['name'],
            description=data['description'],
            rarity=data['rarity'],
            defense=data['defense'],
            is_broken=data.get('is_broken', False)  # Set broken status if provided
        )
        instance._ownership = data.get('ownership', "")  # Set ownership if provided
        return instance

# Subclass: Potion
class Potion(Inventory):
    def __init__(self, name, description, rarity, effect):
        super().__init__(name, description, rarity)
        self.effect = effect
        self.is_used = False

    def to_json(self):
        data = super().to_json()  # Get base item details
        data.update({
            'type': 'Potion',  # Specify the type for JSON
            'effect': self.effect,
            'is_used': self.is_used,
        })
        return data

    @classmethod
    def from_json(cls, data):
        instance = cls(
            name=data['name'],
            description=data['description'],
            rarity=data['rarity'],
            effect=data['effect']
        )
        instance._ownership = data.get('ownership', "")  # Set ownership if provided
        instance.is_used = data.get('is_used', False)  # Set used status if provided
        return instance



#Demo
# Demonstration
if __name__ == "__main__":
    shield = Shield(name="Dragon Shield", description="A sturdy shield forged from dragon scales.", rarity="epic", defense=150)
    print("Original Shield:")
    print(shield)

    # Serialize Shield to JSON
    shield_json = json.dumps(shield.to_json(), indent=4)
    print("\nSerialized Shield JSON:")
    print(shield_json)

    # Deserialize Shield from JSON
    shield_data = json.loads(shield_json)
    restored_shield = Shield.from_json(shield_data)
    print("\nRestored Shield:")
    print(restored_shield)

    # Create a Potion instance
    potion = Potion(name="Healing Potion", description="Restores health over time.", rarity="common", effect="Restore 50 HP")
    print("\nOriginal Potion:")
    print(potion)

    # Serialize Potion to JSON
    potion_json = json.dumps(potion.to_json(), indent=4)
    print("\nSerialized Potion JSON:")
    print(potion_json)

    # Deserialize Potion from JSON
    potion_data = json.loads(potion_json)
    restored_potion = Potion.from_json(potion_data)
    print("\nRestored Potion:")
    print(restored_potion)

Original Shield:
Item: Dragon Shield, Description: A sturdy shield forged from dragon scales., Rarity: epic, Ownership: No ownership.

Serialized Shield JSON:
{
    "name": "Dragon Shield",
    "description": "A sturdy shield forged from dragon scales.",
    "rarity": "epic",
    "ownership": "",
    "type": "Shield",
    "defense": 150,
    "is_broken": false
}

Restored Shield:
Item: Dragon Shield, Description: A sturdy shield forged from dragon scales., Rarity: epic, Ownership: No ownership.

Original Potion:
Item: Healing Potion, Description: Restores health over time., Rarity: common, Ownership: No ownership.

Serialized Potion JSON:
{
    "name": "Healing Potion",
    "description": "Restores health over time.",
    "rarity": "common",
    "ownership": "",
    "type": "Potion",
    "effect": "Restore 50 HP",
    "is_used": false
}

Restored Potion:
Item: Healing Potion, Description: Restores health over time., Rarity: common, Ownership: No ownership.
